In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as kr
from tensorflow import keras as keras
import os , scipy.io
#minors
import math
import time
import scipy
import random
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D ,Conv1D
from tensorflow.keras import regularizers

# plot related
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from tensorflow.keras import backend as K
from scipy.stats import spearmanr

#networking related
import requests
from bs4 import BeautifulSoup

# load dataset from drive
from google.colab import drive
drive.mount('/gdrive')

# linux operation
%cd /gdrive/My Drive/
%cd ML_Data/MM/INCHI/

split_ratio = 0.9 # train data ratio

# load data
X_data = np.genfromtxt('target.dat', delimiter=',', skip_header = 1)
Y_data = np.genfromtxt('label.dat', delimiter=',', skip_header = 1,)
name_list = np.recfromcsv('index.dat')

X_data = 1-X_data
Y_data = 1-Y_data

X_data = X_data.reshape(len(X_data),len(X_data[1]),1)
Y_data = Y_data.reshape(len(Y_data),len(Y_data[1]))

# split data into train and test
x_train = X_data[0:math.floor(split_ratio*len(X_data))]
x_test = X_data[math.floor(split_ratio*len(X_data)):len(X_data)]

y_train = Y_data[0:math.floor(split_ratio*len(Y_data))]
y_test = Y_data[math.floor(split_ratio*len(Y_data)):len(Y_data)]

Mounted at /gdrive
/gdrive/My Drive
/gdrive/My Drive/ML_Data/MM/INCHI


/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:2539: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [ ]:
model = keras.Sequential(name="model_conv1D")
model.add(tf.keras.Input(shape=(len(X_data[1]),1),name='INPUT'))
model.add(keras.layers.Conv1D(filters=2, strides=2, kernel_size=2, activation=keras.layers.LeakyReLU(alpha=0.2),))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv1D(filters=4, strides=2, kernel_size=2, activation=keras.layers.LeakyReLU(alpha=0.2),))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv1D(filters=8, strides=2, kernel_size=2, activation=keras.layers.LeakyReLU(alpha=0.2),))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv1D(filters=16, strides=2, kernel_size=2, activation=keras.layers.LeakyReLU(alpha=0.2),))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv1D(filters=32, strides=2, kernel_size=2, activation=keras.layers.LeakyReLU(alpha=0.2),))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Flatten()) # important
model.add(keras.layers.Dense(64, activation=keras.layers.LeakyReLU(alpha=0.2),kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(keras.layers.Reshape((8,  8)))
model.add(keras.layers.Conv1DTranspose(32, kernel_size=2, strides=2, padding="same", activation=keras.layers.LeakyReLU(alpha=0.2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv1DTranspose(16, kernel_size=2, strides=2, padding="same", activation=keras.layers.LeakyReLU(alpha=0.2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv1DTranspose(8, kernel_size=2, strides=2, padding="same", activation=keras.layers.LeakyReLU(alpha=0.2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv1DTranspose(4, kernel_size=2, strides=2, padding="same", activation=keras.layers.LeakyReLU(alpha=0.2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv1DTranspose(2, kernel_size=2, strides=2, padding="same", activation=keras.layers.LeakyReLU(alpha=0.2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(len(Y_data[1]), name="Dense_output"))
model.compile(optimizer="adam", loss='mean_absolute_error')
model.summary()


In [ ]:
print('START!')
#start = time.time()
# fit with val-set
early_stopping = keras.callbacks.EarlyStopping(patience=150, min_delta=0.0005, restore_best_weights=True)

history = model.fit(x_train,y_train,batch_size=32,epochs=500,validation_split=0.11)
#end = time.time()
print('Train End!')
los=model.evaluate(x_test,y_test)

In [ ]:
# export CSV
train_dat = history.history
loss_dat = train_dat['loss']
val_dat = train_dat['val_loss']
offset = 0

for i in range(len(loss_dat)):
  print(str(i+1+offset)+','+str(loss_dat[i])+','+str(val_dat[i]))

## evaluation

In [ ]:
from scipy.stats import spearmanr
# final evaluation
spearsman_list_MM = []
spearsman_list_NN = []
ans = model.predict(x_test)
for i in range(len(x_test)):
  correlation, p = spearmanr(ans[i],y_test[i])
  spearsman_list_NN.append(correlation)
  correlation, p = spearmanr(x_test[i],y_test[i])
  spearsman_list_MM.append(correlation)

print('Neuron: ',end='')
print(sum(spearsman_list_NN)/len(x_test))
print('Molecular Mechanics: ',end='')
print(sum(spearsman_list_MM)/len(x_test))

# Get list of spearsman coefficient

In [ ]:
for i in range(len(x_test)):
  print(str(i+1)+',',end='')
  print(spearsman_list_NN[i])

## Draw IR spectra

In [ ]:
def get_spearsman_ir(i):
  los=model.evaluate(x_test[i].reshape(1,len(x_test[1]),1),y_test[i].reshape(1,len(x_test[1])))
  print('loss : ',end='')
  print(los)
  ans = model.predict(x_test[i].reshape(1,len(x_test[1]),1))#.reshape(1,len(x_test[1])))
  ans = ans.reshape(len(x_test[1]))
  from scipy.stats import spearmanr
  correlation, p = spearmanr(ans,y_test[i])
  print('Correlation:', correlation)
  # print('p-value:', p)
  return  correlation

def get_name(cas):
  re = re = requests.get('https://webbook.nist.gov/cgi/cbook.cgi?Name='+str(cas)+'&Units=SI')
  soup = BeautifulSoup(re.text, 'html.parser')
  check_tag = soup.find(id='Top')
  if check_tag and check_tag.get_text() != 'Name Not Found':
    return check_tag.text

def plotter(plt_list):
  plt.rcParams["figure.figsize"] = (15,2.5*len(plt_list))
  for i in range(len(plt_list)):
    fig =plt.subplot(len(plt_list),1,i+1)
    #model validation
    #k=randomlist[6]
    k = plt_list[i]
    ans = model.predict(x_test[k].reshape(1,len(X_data[1]),1))

    answer = ans.reshape(len(X_data[1]))
    answer=1-answer
    range_ir = np.arange(400, 4001, 2)
    fig.plot(range_ir, 1-x_test[k],  c='g', label='DB' , linewidth=1.0)
    # scipy.signal.savgol_filter(1-Y_data[k],25,7)
    fig.plot(range_ir, 1-y_test[k] ,  c='b', label='DB' , linewidth=1.0)
    fig.plot(range_ir, answer ,  c='r', label='DB' , linewidth=1.0)
    #y_smooth = scipy.signal.savgol_filter((1-Y_data[k]).reshape(3151),window_length=25,polyorder=3,delta=1,)
    #fig.plot(range(600, 3751), y_smooth ,  c='r', label='DB' , linewidth=1.0)

    fig.invert_xaxis()
    fig.grid(True)
    #fig.title.set_text(name_list[k+len(x_train)].tostring('c').strip(b'\x00'))
    name_cas = str(name_list[k+len(x_train)].tostring('c').strip(b'\x00'),encoding='utf-8')
    print(name_cas)
    get_spearsman_ir(k)
    plt.title(get_name(name_cas))
    print(get_name(name_cas))
    fig.set_ylim([0, 1.2])
    #plt.show()


#plotter([69,70,71,72])
plotter(random.sample(range(len(x_test)),1))
# plotter([493,223,207])

# Save Model

In [ ]:
model.save('CNN/CNN_Model.h5')

# Load Model

In [ ]:
model  = tf.keras.models.load_model('CNN/CNN_Model.h5')